In [1]:

import numpy as np
import time
from pylab import *
import copy
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

# Set the transformation for each dataset folder
transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Import the dataset
data_dir = 'drive/My Drive/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transforms[x])
                  for x in ['train', 'val', 'test']}

# Shuffle the dataset and create batches from the dataset
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}

# Get the number of images in each folder
data_size = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

# Get the class name
class_names = image_datasets['train'].classes

# Enable the GPU if it exists
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Load the model (Download only at first time loading the model)
model_resnet = models.resnet18(pretrained=True)
model_vgg16 = models.vgg16(pretrained=True)
model_alexnet = models.alexnet(pretrained=True)

# Define the function to train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

# Set the requires_grad on each parameter to false, 
# so it will not calculate the gradients
for param in model_vgg16.parameters():
    param.requires_grad = False

for param in model_resnet.parameters():
    param.requires_grad = False

for param in model_alexnet.parameters():
    param.requires_grad = False

# Set the new final layer for our new dataset
num_ftrs = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(num_ftrs, len(class_names))

num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, len(class_names))

num_ftrs = model_alexnet.classifier[6].in_features
model_alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names))

# Enable GPU for the model
model_vgg16 = model_vgg16.to(device)
model_resnet = model_resnet.to(device)
model_alexnet = model_alexnet.to(device)

# Set the loss function
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [5]:
print("---------------------VGG16 MODEL------------------------")
print(" ")
# Set the optimizer and the scheduler to update the weights, and train the model.
optimizer_conv = optim.SGD(model_vgg16.classifier[6].parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_vgg16 = train_model(model_vgg16, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

---------------------VGG16 MODEL------------------------
 
Epoch 0/24
----------
train Loss: 0.4761 Acc: 0.8278
val Loss: 0.2237 Acc: 0.8980

Epoch 1/24
----------
train Loss: 0.4375 Acc: 0.8462
val Loss: 0.1318 Acc: 0.9490

Epoch 2/24
----------
train Loss: 0.4313 Acc: 0.8510
val Loss: 0.1463 Acc: 0.9451

Epoch 3/24
----------
train Loss: 0.4200 Acc: 0.8624
val Loss: 0.1210 Acc: 0.9529

Epoch 4/24
----------
train Loss: 0.4279 Acc: 0.8681
val Loss: 0.0957 Acc: 0.9608

Epoch 5/24
----------
train Loss: 0.4162 Acc: 0.8747
val Loss: 0.1398 Acc: 0.9451

Epoch 6/24
----------
train Loss: 0.4008 Acc: 0.8760
val Loss: 0.0629 Acc: 0.9804

Epoch 7/24
----------
train Loss: 0.3715 Acc: 0.8821
val Loss: 0.0596 Acc: 0.9804

Epoch 8/24
----------
train Loss: 0.3315 Acc: 0.8988
val Loss: 0.0547 Acc: 0.9843

Epoch 9/24
----------
train Loss: 0.3033 Acc: 0.8935
val Loss: 0.0452 Acc: 0.9843

Epoch 10/24
----------
train Loss: 0.3433 Acc: 0.8808
val Loss: 0.0600 Acc: 0.9765

Epoch 11/24
----------
trai

In [6]:
print("---------------------RES NET MODEL------------------------")
print(" ")
optimizer_conv = optim.SGD(model_resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_resnet = train_model(model_resnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

---------------------RES NET MODEL------------------------
 
Epoch 0/24
----------
train Loss: 0.6762 Acc: 0.7463
val Loss: 0.1261 Acc: 0.9529

Epoch 1/24
----------
train Loss: 0.6250 Acc: 0.7699
val Loss: 0.0851 Acc: 0.9686

Epoch 2/24
----------
train Loss: 0.4899 Acc: 0.8203
val Loss: 0.0538 Acc: 0.9804

Epoch 3/24
----------
train Loss: 0.4895 Acc: 0.8225
val Loss: 0.0343 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.5049 Acc: 0.8287
val Loss: 0.0623 Acc: 0.9725

Epoch 5/24
----------
train Loss: 0.4581 Acc: 0.8295
val Loss: 0.0593 Acc: 0.9882

Epoch 6/24
----------
train Loss: 0.4084 Acc: 0.8462
val Loss: 0.0578 Acc: 0.9922

Epoch 7/24
----------
train Loss: 0.3653 Acc: 0.8659
val Loss: 0.0757 Acc: 0.9804

Epoch 8/24
----------
train Loss: 0.3732 Acc: 0.8585
val Loss: 0.0676 Acc: 0.9647

Epoch 9/24
----------
train Loss: 0.3919 Acc: 0.8633
val Loss: 0.0514 Acc: 0.9804

Epoch 10/24
----------
train Loss: 0.3404 Acc: 0.8685
val Loss: 0.0484 Acc: 0.9882

Epoch 11/24
----------
tr

In [8]:
print("---------------------ALEX NET MODEL------------------------")
print(" ")
optimizer_conv = optim.SGD(model_alexnet.classifier[6].parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_alexnet = train_model(model_alexnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

---------------------ALEX NET MODEL------------------------
 
Epoch 0/24
----------
train Loss: 0.7169 Acc: 0.8146
val Loss: 0.2797 Acc: 0.9333

Epoch 1/24
----------
train Loss: 0.7382 Acc: 0.8532
val Loss: 0.3382 Acc: 0.9373

Epoch 2/24
----------
train Loss: 0.6711 Acc: 0.8720
val Loss: 0.2314 Acc: 0.9529

Epoch 3/24
----------
train Loss: 0.6980 Acc: 0.8747
val Loss: 0.2540 Acc: 0.9412

Epoch 4/24
----------
train Loss: 0.6496 Acc: 0.8712
val Loss: 1.1210 Acc: 0.7843

Epoch 5/24
----------
train Loss: 0.5737 Acc: 0.8913
val Loss: 0.1782 Acc: 0.9608

Epoch 6/24
----------
train Loss: 0.6052 Acc: 0.8843
val Loss: 0.1943 Acc: 0.9686

Epoch 7/24
----------
train Loss: 0.3937 Acc: 0.9137
val Loss: 0.1818 Acc: 0.9686

Epoch 8/24
----------
train Loss: 0.4005 Acc: 0.9067
val Loss: 0.1813 Acc: 0.9608

Epoch 9/24
----------
train Loss: 0.3026 Acc: 0.9220
val Loss: 0.2039 Acc: 0.9569

Epoch 10/24
----------
train Loss: 0.3079 Acc: 0.9251
val Loss: 0.1994 Acc: 0.9490

Epoch 11/24
----------
t

In [9]:
# Show the images and the ground truth
dataiter = iter(dataloaders['test'])
images, labels = dataiter.next()

print('GroundTruth: ', ', '.join('%5s' % class_names[x] for x in labels))

# Predict the model
images = images.to(device)
labels = images.to(device)
output = model_resnet(images)
_, predicted = torch.max(output, 1)
print('Predicted: ', ', '.join('%5s' % class_names[x] for x in predicted))

GroundTruth:  Cedar Apple Rust, Healthy, Healthy, Healthy
Predicted:  Cedar Apple Rust, Healthy, Healthy, Healthy


In [10]:
# Save The Model 
PATH = './fix_resnet18.pth'
torch.save(model_resnet.state_dict(), PATH)

In [12]:
%run -i 'app.py'

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
